In [2]:
import pickle
with open('phn_sim.pkl', 'rb') as file:
    similarity_dict = pickle.load(file)
print(similarity_dict)

{('OW', 'UW'): 0.495, ('OW', 'EY'): 0.019, ('OW', 'AW'): 0.338, ('OW', 'AH'): 0.143, ('OW', 'AO'): 0.492, ('OW', 'AY'): 0.021, ('OW', 'EH'): 0.134, ('OW', 'K'): 0.044, ('OW', 'NG'): 0.095, ('OW', 'F'): 0.043, ('OW', 'JH'): 0.027, ('OW', 'M'): 0.101, ('OW', 'CH'): 0.04, ('OW', 'IH'): 0.146, ('OW', 'UH'): 0.139, ('OW', 'HH'): 0.042, ('OW', 'L'): 0.189, ('OW', 'AA'): 0.171, ('OW', 'R'): 0.184, ('OW', 'TH'): 0.042, ('OW', 'AE'): 0.095, ('OW', 'D'): 0.052, ('OW', 'Z'): 0.07, ('OW', 'OY'): 0.31, ('OW', 'DH'): 0.033, ('OW', 'IY'): 0.137, ('OW', 'B'): 0.022, ('OW', 'W'): 0.152, ('OW', 'S'): 0.076, ('OW', 'T'): 0.071, ('OW', 'SH'): 0.037, ('OW', 'ZH'): 0.043, ('OW', 'ER'): 0.121, ('OW', 'V'): 0.045, ('OW', 'Y'): 0.2, ('OW', 'N'): 0.081, ('OW', 'G'): 0.016, ('OW', 'P'): 0.046, ('UW', 'EY'): 0.156, ('UW', 'AW'): 0.499, ('UW', 'AH'): 0.329, ('UW', 'AO'): 0.3, ('UW', 'AY'): 0.173, ('UW', 'EH'): 0.299, ('UW', 'K'): 0.21, ('UW', 'NG'): 0.244, ('UW', 'F'): 0.192, ('UW', 'JH'): 0.224, ('UW', 'M'): 0.26

In [4]:
phoneme_list = ['OW', 'UW', 'EY', 'AW', 'AH', 'AO', 'AY', 'EH', 'K', 'NG', 'F', 'JH', 
         'M',  'CH', 'IH', 'UH', 'HH', 'L',  'AA', 'R', 'TH', 'AE', 'D', 'Z',
         'OY', 'DH', 'IY', 'B',  'W',  'S',  'T', 'SH', 'ZH', 'ER', 'V', 'Y',
         'N',  'G',  'P']
high_risk = ["P", "B", "T", "D", "K", "G", "F", "V", "TH", "DH", "S", "Z", "CH", "JH"]
med_risk = ["M", "N", "NG", "L", "R", "W", "Y"]
low_risk = ["AA", "AE", "AH", "AO", "AW", "AY", "EH", "ER", "EY", "IH", "IY", "OW", "OY", "UH", "UW"]

In [10]:
import pronouncing
import numpy as np
import copy
import random
import re

TYPE = ("REP","SUB","MIXED","POLY","DEL")

def sentence_to_phonemes(sentence):
    words = sentence.split()
    phonemes = [
        [re.sub(r'\d', '', phoneme) for phoneme in pronouncing.phones_for_word(word.lower())[0].split()]
        for word in words if pronouncing.phones_for_word(word.lower())
    ]
    return " ".join(phoneme for sublist in phonemes for phoneme in sublist)

SENT = []
REF = []
SRC = []
LABEL = []
NUM = []

with open('text/pre_VCTK.txt', 'r', encoding='utf-8') as file:
    for line in file:
        sentence = line.strip()
        if "should be doing that for herThey have" in sentence or "falls upon the green of the first" in sentence:
            continue
        ref_pho = sentence_to_phonemes(sentence).split(" ")
        idx = list(range(len(ref_pho)))
        generate_time = random.randint(45, 50)
        for t in range(generate_time):
            src_pho = copy.deepcopy(ref_pho)
            label = [1] * len(ref_pho)

            dysfluent_times = random.randint(1, 3)
            dysfluent_pb = []
            for i in ref_pho:
                if i in high_risk:
                    dysfluent_pb.append(0.5)
                elif i in med_risk:
                    dysfluent_pb.append(0.3)
                else:
                    dysfluent_pb.append(0.2)
            sample_result = np.random.choice(idx, size = dysfluent_times, replace = False, p = dysfluent_pb/np.sum(dysfluent_pb))
            sample_result = np.sort(sample_result)
            dysfluent_pho = np.array(ref_pho)[sample_result]
            for j in range(len(sample_result)):
                target_pho = dysfluent_pho[j]
                target_idx = sample_result[j]
                dysfluent_type = random.choices(TYPE, weights = [0.2, 0.35, 0.45, 0.2, 0.3], k = 1)[0]
                    
                if dysfluent_type == "POLY":
                    tripple = random.randint(0, 1)
                    if tripple == 0:
                        flg = False
                        if j == len(sample_result) - 1:
                            if target_idx + 1 < len(src_pho): flg = True
                        else:
                            if sample_result[j + 1] - target_idx != 1: flg = True
                        if flg:
                            src_pho.insert(target_idx, src_pho[target_idx + 1])
                            src_pho.insert(target_idx, target_pho)
                            label.insert(target_idx, 0)
                            label.insert(target_idx, 0)
                            for m in range(len(sample_result)):
                                if sample_result[m] >= target_idx: sample_result[m] += 2
                        else: dysfluent_type = "REP"
                    else:
                        flg = False
                        if j == len(sample_result) - 1:
                            if target_idx + 2 < len(src_pho): flg = True
                        else:
                            if sample_result[j + 1] - target_idx > 2: flg = True
                        if flg:
                            src_pho.insert(target_idx, src_pho[target_idx + 2])
                            src_pho.insert(target_idx, src_pho[target_idx + 2])
                            src_pho.insert(target_idx, target_pho)
                            label.insert(target_idx, 0)
                            label.insert(target_idx, 0)
                            label.insert(target_idx, 0)
                            for m in range(len(sample_result)):
                                if sample_result[m] >= target_idx: sample_result[m] += 3
                        else: dysfluent_type = "REP"

                if dysfluent_type == "REP":
                    repetition_time = random.randint(1, 2)
                    for k in range(repetition_time):
                        src_pho.insert(target_idx, target_pho)
                        label.insert(target_idx, 0)
                    for m in range(len(sample_result)):
                        if sample_result[m] >= target_idx: sample_result[m] += repetition_time

                elif dysfluent_type == "SUB":
                    sub_pb = []         
                    for k in phoneme_list:
                        if (target_pho, k) in list(similarity_dict.keys()):
                            sub_pb.append(similarity_dict[(target_pho, k)])
                        elif (k, target_pho) in list(similarity_dict.keys()):
                            sub_pb.append(similarity_dict[(k, target_pho)])
                        else: sub_pb.append(0)
                    sub_pho = random.choices(phoneme_list, weights = sub_pb, k = 1)[0]
                    src_pho[target_idx] = sub_pho

                elif dysfluent_type == "DEL":
                    label[target_idx] = 2
                    src_pho[target_idx] = "[None]"
 
                elif dysfluent_type == "MIXED":
                    mix_pb = []

                    for k in phoneme_list:
                        if (target_pho, k) in list(similarity_dict.keys()):
                            mix_pb.append(similarity_dict[(target_pho, k)])
                        elif (k, target_pho) in list(similarity_dict.keys()):
                            mix_pb.append(similarity_dict[(k, target_pho)])
                        if k == target_pho: mix_pb.append(0)

                    mix_pho1 = random.choices(phoneme_list, weights = mix_pb, k = 1)[0]
                    mix_pho2 = random.choices(phoneme_list, weights = mix_pb, k = 1)[0]

                    if (mix_pho1, target_pho) in similarity_dict: sim_pho1 = similarity_dict[(mix_pho1, target_pho)]
                    else: sim_pho1 = similarity_dict[(target_pho, mix_pho1)]

                    if (mix_pho2, target_pho) in similarity_dict: sim_pho2 = similarity_dict[(mix_pho2, target_pho)]
                    else: sim_pho2 = similarity_dict[(target_pho, mix_pho2)]

                    if sim_pho1 < sim_pho2: 
                        temp = mix_pho1
                        mix_pho1 = mix_pho2
                        mix_pho2 = temp

                    flg = random.choices(population = [0, 1], weights = [0.75, 0.25], k = 1)[0]
                    if flg == 0:
                        if target_idx + 1 < len(src_pho):
                            if (mix_pho1, target_pho) in similarity_dict: sim1 = similarity_dict[(mix_pho1, target_pho)]
                            else: sim1 = similarity_dict[(target_pho, mix_pho1)]
                            if (mix_pho1, src_pho[target_idx + 1]) in similarity_dict: sim2 = similarity_dict[(mix_pho1, src_pho[target_idx + 1])]
                            elif (src_pho[target_idx + 1], mix_pho1) in similarity_dict: sim2 = similarity_dict[(src_pho[target_idx + 1], mix_pho1)]
                            else: sim2 = 1
                            if sim1 >= sim2: label.insert(target_idx, 0)
                            else: label.insert(target_idx + 1, 0)
                        else:
                            label.insert(target_idx, 0)
                        src_pho.insert(target_idx + 1, mix_pho1)
                    elif flg == 1:
                        src_pho[target_idx] = mix_pho1
                        if target_idx + 1 < len(src_pho):
                            if mix_pho1 == mix_pho2: sim1 = 1
                            elif (mix_pho2, mix_pho1) in similarity_dict: sim1 = similarity_dict[(mix_pho2, mix_pho1)]
                            else: sim1 = similarity_dict[(mix_pho1, mix_pho2)]
                            if mix_pho2 == src_pho[target_idx + 1]: sim2 = 1
                            elif (mix_pho2, src_pho[target_idx + 1]) in similarity_dict: sim2 = similarity_dict[(mix_pho2, src_pho[target_idx + 1])]
                            else: sim2 = similarity_dict[(src_pho[target_idx + 1], mix_pho2)]
                            if sim1 >= sim2: label.insert(target_idx, 0)
                            else: label.insert(target_idx + 1, 0)
                        else:
                            label.insert(target_idx, 0)
                        src_pho.insert(target_idx + 1, mix_pho2)
                    for m in range(len(sample_result)):
                        if sample_result[m] > target_idx: sample_result[m] += 1


            SENT.append(sentence)
            REF.append(" ".join(ref_pho))
            src_pho = list(filter(lambda x: x != "[None]", src_pho))
            SRC.append(" ".join(src_pho))
            label.append(3)
            LABEL.append(" ".join([str(i) for i in label]))
            NUM.append(dysfluent_times)


In [12]:
import json

data = []

for idx, (stn, ref, src, label, num) in enumerate(zip(SENT, REF, SRC, LABEL, NUM)):
    data.append({
        "id": idx+1,
        "sentence": stn,
        "REF": ref,
        "SRC": src,
        "LABEL": label,
        "num": num
    })

with open('train_data/data7_del.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

In [14]:
with open('train_data/data7_del.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

last_three = data[-6:-3]

print(last_three)

[{'id': 2134570, 'sentence': 'I enjoyed a new novel', 'REF': 'AY EH N JH OY D AH N UW N AA V AH L', 'SRC': 'AY EH N JH OY D AH N UW N AA F P AH L', 'LABEL': '1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 3', 'num': 1}, {'id': 2134571, 'sentence': 'I enjoyed a new novel', 'REF': 'AY EH N JH OY D AH N UW N AA V AH L', 'SRC': 'AY EH N JH OY D CH AH N UW AA AA V EY L', 'LABEL': '1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 3', 'num': 3}, {'id': 2134572, 'sentence': 'I enjoyed a new novel', 'REF': 'AY EH N JH OY D AH N UW N AA V AH L', 'SRC': 'AY EH N JH OY D D D N UW N AA B AH L', 'LABEL': '1 1 1 1 1 0 0 1 2 1 1 1 1 1 1 1 3', 'num': 3}]
